In [1]:
import datetime
import sqlite3 as lite
import requests
import pandas as pd
from collections import defaultdict

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=30)

api_key = "3c2c05f7bb4af88f75f72dfa3c9540ee"
url = 'https://api.forecast.io/forecast/' + api_key

cities = { "Atlanta": '33.762909,-84.422675',
            "LV": '36.229214,-115.26008',
            "SF": '37.727239,-123.032229',
            "Buffalo": '42.886448,-78.878372',
            "NYC": '40.663619,-73.938589'
        }

In [2]:
start_date.strftime('%Y-%m-%dT12:00:00')
#start_date.strftime('%s')

'2016-06-27T12:00:00'

In [3]:
con = lite.connect('weather.db')

cur = con.cursor()

In [4]:
cities.keys()
with con:
    cur.execute('DROP TABLE IF EXISTS daily_temp')
    cur.execute('CREATE TABLE daily_temp (day_of_reading INT, Atlanta REAL, LV REAL, SF REAL, Buffalo REAL, NYC REAL);')

In [5]:
query_date = start_date

with con:
    while query_date < end_date:
        cur.execute('INSERT INTO daily_temp(day_of_reading) VALUES (?)', (int(query_date.strftime('%d')),))
        query_date += datetime.timedelta(days=1)

In [6]:
r = requests.get(url+'/42.886448,-78.878372'+','+query_date.strftime('%Y-%m-%dT12:00:00'))
r.json()['daily']['data'][0]['temperatureMax']

84.5

In [9]:
for k,v in cities.items():
    query_date = start_date
    while query_date < end_date:
        #API query
        r = requests.get(url+'/'+v+','+query_date.strftime('%Y-%m-%dT12:00:00'))
        max_temp = r.json()['daily']['data'][0]['temperatureMax']
        
        with con:
            #insert the temperature max to the database
            cur.execute('UPDATE daily_temp SET ' + k + ' = ' + str(max_temp) + ' WHERE day_of_reading = ' + query_date.strftime('%d'))
            
        query_date += datetime.timedelta(days=1)
        
con.close()

In [10]:
con = lite.connect('weather.db')
cur = con.cursor()

#Reading sql query and putting it in a dataframe
df = pd.read_sql_query("SELECT * FROM daily_temp ORDER BY day_of_reading",con,index_col='day_of_reading')

In [11]:
df.head()

,Atlanta,LV,SF,Buffalo,NYC
day_of_reading,,,,,
1,93.65,95.91,53.76,72.69,77.23
2,96.21,94.56,55.00,71.03,78.48
3,96.66,102.60,54.76,76.03,79.57
4,94.27,104.99,55.38,81.15,79.91
5,85.16,104.19,54.76,76.37,89.00


In [17]:
temp_diff_df = pd.DataFrame()

In [39]:
city_list = []
for i in df.columns:
    city_list.append(i)
for city in city_list:
    temp_diff = df[city].max() - df[city].min()
    print ("Maximum temperature difference for "+ city +" in the recent month is " + str(temp_diff)+" F.\n")

Maximum temperature difference for Atlanta in the recent month is 11.65 F.

Maximum temperature difference for LV in the recent month is 13.72 F.

Maximum temperature difference for SF in the recent month is 4.91 F.

Maximum temperature difference for Buffalo in the recent month is 18.03 F.

Maximum temperature difference for NYC in the recent month is 23.51 F.



In [31]:
temp_diff_df

,Atlanta,LV,SF,Buffalo,NYC
